<a href="https://colab.research.google.com/github/ShreeyaKokate/Twitter-Analysis/blob/main/Tweepy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tweepy
from tweepy import Cursor
from datetime import datetime, date, time
import json
import pandas as pd
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib as mpl
import csv
import matplotlib.pyplot as plt

import operator
from textblob import TextBlob
from textblob import Word
from textblob.sentiments import NaiveBayesAnalyzer

In [2]:
#Authentication
consumer_key = '5k6YW9hGYcmjZQKA3SgV8WaKj'
consumer_secret = 'qiFWNQHwakkZiWlRIaDfd05zq7hJ6d8T4TLLAi08ylRhZwNizx'
access_key = '1082654918987694083-DhNd1zerKLI5lvlTdCaTIhkK8Xm1gA'
access_secret = 'CHYHMUm7wwdfnYPZzbpfsTo9BMvk9nMIwga0pqChJG1aM'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret) #Interacting with twitter's API
auth.set_access_token(access_key, access_secret)
api = tweepy.API (auth) #creating the API object


In [3]:
#Extracting Tweets
results = []
for tweet in tweepy.Cursor (api.search, q='Black Lives Matter', lang = "en").items(2000): 
    results.append(tweet)
    
print (type(results))
print (len(results))

<class 'list'>
2000


In [4]:
#Store tweets data in a dataframe

def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    
    return data_set
data_set = tweets_df(results)



In [5]:
print(data_set)

                       id  ...                                           Hashtags
0     1351434471522914310  ...                                                 []
1     1351434422390648833  ...                                                 []
2     1351434413070958592  ...  [{'text': 'PS5Share', 'indices': [37, 46]}, {'...
3     1351434355873341442  ...                                                 []
4     1351434338957627394  ...                                                 []
...                   ...  ...                                                ...
1995  1351388831220600832  ...             [{'text': 'OtD', 'indices': [21, 25]}]
1996  1351388810790187008  ...                                                 []
1997  1351388804486197251  ...                                                 []
1998  1351388773112827904  ...                                                 []
1999  1351388765785362433  ...                                                 []

[2000 rows x 8 

In [6]:
data_set.to_csv("/content/drive/MyDrive/Colab Notebooks/BLM1.csv")

In [8]:
# Remove tweets with duplicate text

text = data_set["text"]

for i in range(0,len(text)):
    txt = ' '.join(word for word in text[i] .split() if not word.startswith('https:'))
    data_set.at[i, 'text2']= txt
    
data_set.drop_duplicates('text2', inplace=True)
data_set.reset_index(drop = True, inplace=True)
data_set.drop('text', axis = 1, inplace = True)
data_set.rename(columns={'text2': 'text'}, inplace=True)

In [9]:
text = data_set["text"]

for i in range(0,len(text)):
    textB = TextBlob(text[i])
    sentiment = textB.sentiment.polarity
    data_set.at[i, 'Sentiment'] =sentiment
    if sentiment <0.00:
        SentimentClass = 'Negative'
        data_set.at[i, 'SentimentClass'] = SentimentClass 
    elif sentiment >0.00:
        SentimentClass = 'Positive'
        data_set.at[i, 'SentimentClass']= SentimentClass 
    else:
        SentimentClass = 'Neutral'
        data_set.at[i, 'SentimentClass']= SentimentClass 

In [10]:
print(textB)

@awkward_duck @MsBraxtonTweets Video - BLM AT SCHOOL Black Lives Matter at School · Racial Justice


In [11]:
data_set.to_csv("/content/drive/MyDrive/Colab Notebooks/BLM1-1.csv")

In [12]:
Htag_df = pd.DataFrame()
j = 0

for tweet in range(0,len(results)):
    hashtag = results[tweet].entities.get('hashtags')
    for i in range(0,len(hashtag)):
        Htag = hashtag[i]['text'] 
        Htag_df.at[j, 'Hashtag']= Htag
        j = j+1

In [13]:
print(Htag_df)

                          Hashtag
0                        PS5Share
1    MarvelsSpiderManMilesMorales
2              RhythminTheMorning
3              WhereHitMusicLives
4                BlackLivesMatter
..                            ...
187                        MLKDay
188                        MLKDay
189                           BLM
190                           OtD
191                           OtD

[192 rows x 1 columns]


In [15]:
Htag_wordcloud = Htag_df.groupby('Hashtag').size()